In [64]:
# Importing modules
import pandas as pd
import os
os.chdir('..')
# Read data into papers
papers = pd.read_csv("C:/Users/nicyj/becode/KPMG/NEW/metadata_enriched.csv")
# Print head
papers.head()

,Unnamed: 0.1,Unnamed: 0,deposit_number,jc_number,Omschrijving,Categorie,jc_name_fr,jc_name_nl,cla_title_en,cla_title_nl,...,TEXT,Summary,keywords,toepassing,loon,duur,premie,Werkgever,werkloosheid,bonus
0,0,0,175235,100.0,Aanvullend Paritair Comité voor de werklieden,Paritaire organen bevoegd voor de werknemers d...,auxiliary joint committee for workers,aanvullend paritair comite voor de werklieden,Minimum hourly wage,Minimumurloon,...,COLLECTIEVE ARBEIDSOVEREENKOMST GESLOTEN OP 21...,Artikel 7 Het in artikel 3 bepaalde minimum uu...,['premies'],COLLECTIEVE ARBEIDSOVEREENKOMST GESLOTEN OP 21...,NaN,Deze overeenkomst is gesloten voor onbepaalde ...,NaN,NaN,NaN,NaN
1,1,3,172224,100.0,Aanvullend Paritair Comité voor de werklieden,Paritaire organen bevoegd voor de werknemers d...,auxiliary joint committee for workers,aanvullend paritair comite voor de werklieden,Credit time 2021-2022,Time credit 2021-2022,...,"SERVICE PUBLIC FEDERAL Emploi, Travail et Conc...","SERVICE PUBLIC FEDERAL Emploi, Travail et Conc...",[],: 26/04/2022 N°: 172224/CO/100 Deze collectiev...,NaN,Hoofdstuk IV: Duur van de overeenkomst Art.,NaN,NaN,NaN,NaN
2,2,5,171208,100.0,Aanvullend Paritair Comité voor de werklieden,Paritaire organen bevoegd voor de werknemers d...,auxiliary joint committee for workers,aanvullend paritair comite voor de werklieden,Unemployment scheme with company supplement - ...,Unemployment scheme with company supplement - ...,...,COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7 DECEMBER...,HOOFDSTUK IV – TOEPASSINGSREGELS Artikel 6 §1 ...,[],Artikel 5 In geval van geschil betreffende de ...,NaN,HOOFDSTUK V - INWERKINGTREDING EN DUUR VAN DE ...,NaN,Het ontwerp van toetredingsakte wordt door de ...,COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7 DECEMBER...,NaN
3,3,6,171209,100.0,Aanvullend Paritair Comité voor de werklieden,Paritaire organen bevoegd voor de werknemers d...,auxiliary joint committee for workers,aanvullend paritair comite voor de werklieden,Unemployment scheme with company supplement fo...,Unemployment scheme with company supplement fo...,...,COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7 DECEMBER...,UITVOERING VAN ARTIKEL 4 VAN DE COLLECT DECEMB...,[],Artikel 5 In geval van geschil betreffende de ...,NaN,HOOFDSTUK V - INWERKINGTREDING EN DUUR VAN DE ...,NaN,Het ontwerp van toetredingsakte wordt door de ...,COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7 DECEMBER...,NaN
4,4,7,170863,100.0,Aanvullend Paritair Comité voor de werklieden,Paritaire organen bevoegd voor de werknemers d...,auxiliary joint committee for workers,aanvullend paritair comite voor de werklieden,Purchasing power - Annual premium,Purchasing Power - Annual Premium,...,COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7 DECEMBER...,Overeenkomstig artikel 14 van de wet van 5 dec...,[],NaN,NaN,3 Onderhavige collectieve arbeidsovereenkomst ...,De te vervullen voorwaarden zijn de volgende: ...,NaN,NaN,NaN


In [65]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = \
papers['TEXT'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

<>:5: DeprecationWarning: invalid escape sequence '\.'
<>:5: DeprecationWarning: invalid escape sequence '\.'
C:\Users\nicyj\AppData\Local\Temp\ipykernel_18340\3112138311.py:5: DeprecationWarning: invalid escape sequence '\.'
  papers['TEXT'].map(lambda x: re.sub('[,\.!?]', '', x))


0    collectieve arbeidsovereenkomst gesloten op 21...
1    service public federal emploi travail et conce...
2    collectieve arbeidsovereenkomst van 7 december...
3    collectieve arbeidsovereenkomst van 7 december...
4    collectieve arbeidsovereenkomst van 7 december...
Name: paper_text_processed, dtype: object

In [66]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy

stop_words = nltk.corpus.stopwords.words('dutch')
lists = ['la','le','art','du','les','co','et','december','per', 'des', 'juni', 'nr','vanaf','au','januari','gesloten','artikel','collectieve','jaar',
        'fonds', 'hoofdstuk', 'paritair','dag','comité', 'besluit','solidarite', 'regime', 'bijdrage', 'cao', 'dag', 'ou', 'est', 'aanvullen','koninklijk',
        'volgen', 'categorie', 'onderneming', 'periode', 'houden','sectoraal', 'arbeider']
stop_words.extend(lists)
print(stop_words)

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words_nostops = remove_stopwords(data_words)
print(data_words[:1][0][:30])

data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('nl_core_news_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:4]) #it will print the lemmatized data.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicyj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere', 'la', 'le', 'art', 'du', 'les', 'co', 'et', 'december', 'per', 'des', 'juni', 'nr', 'vanaf', 'au', 'januari', 'gesloten', 'artikel', 'collectieve', 'jaar', 'fonds', 'hoofdstuk', 'paritair', 'dag', 'comité', 'besluit', 'solidarite', 'regime', '

In [67]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 3), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 3), (22, 3), (23, 1), (24, 1), (25, 1), (26, 6), (27, 1), (28, 2), (29, 1)]


In [71]:
from pprint import pprint
# number of topics
num_topics = 20

for i in range(num_topics):
    
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=i+1)
    # Print the Keyword in the 10 topics
    #pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    print(i+1)
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))

1

Perplexity:  -7.677709639635942
2

Perplexity:  -7.6773959701493455
3

Perplexity:  -7.680149835003462
4

Perplexity:  -7.7112830492267115
5

Perplexity:  -7.708374294104414
6

Perplexity:  -7.69976797761401
7

Perplexity:  -7.736235371560818
8

Perplexity:  -7.739732234784205
9

Perplexity:  -7.7492588999804175
10

Perplexity:  -7.80186903989962
11

Perplexity:  -7.823633120715083
12

Perplexity:  -7.788423642319992
13

Perplexity:  -7.858378713276002
14

Perplexity:  -7.85297529681718
15

Perplexity:  -7.867860735061894
16

Perplexity:  -7.877396576653605
17

Perplexity:  -7.9118814411508875
18

Perplexity:  -7.942517500524381
19

Perplexity:  -7.938993719542403
20

Perplexity:  -7.93153674797879


In [72]:
from pprint import pprint
# number of topics
num_topics = 18

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"werknemer" + 0.016*"werkgever" + 0.014*"sociaal" + 0.009*"betreffen" '
  '+ 0.009*"toepassing" + 0.008*"comité" + 0.005*"recht" + 0.005*"kader" + '
  '0.005*"dag" + 0.004*"duur"'),
 (1,
  '0.026*"werknemer" + 0.013*"werkgever" + 0.009*"betreffen" + 0.008*"sociaal" '
  '+ 0.006*"bedrijfstoeslag" + 0.006*"toepassing" + 0.006*"stelsel" + '
  '0.006*"recht" + 0.006*"maand" + 0.006*"bepalen"'),
 (2,
  '0.009*"werkgever" + 0.008*"sociaal" + 0.008*"werknemer" + 0.008*"recht" + '
  '0.007*"betreffen" + 0.007*"arbeider" + 0.007*"wet" + 0.006*"bedoelen" + '
  '0.006*"toepassing" + 0.005*"vergoeding"'),
 (3,
  '0.024*"werknemer" + 0.015*"toepassing" + 0.013*"werkgever" + '
  '0.012*"betreffen" + 0.012*"werkloosheid" + 0.010*"bedrijfstoeslag" + '
  '0.009*"stelsel" + 0.009*"sociaal" + 0.008*"bepalen" + 0.008*"aanvullen"'),
 (4,
  '0.012*"betreffen" + 0.009*"werkgever" + 0.009*"comité" + 0.008*"sociaal" + '
  '0.007*"toepassing" + 0.007*"werknemer" + 0.007*"bepalen" + 0.005*"arbeider

In [73]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.8904978294196075


In [74]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = "C:/Users/nicyj/becode/KPMG/NEW/ldavis_prepared.html"
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, "C:/Users/nicyj/becode/KPMG/NEW/ldavis_prepared.html")
LDAvis_prepared

C:\Users\nicyj\becode\KPMG\cla\Lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.045115 -0.010171       1        1  19.595847
6     -0.056432 -0.000162       2        1  15.708717
13    -0.011176  0.023900       3        1  14.124132
5     -0.013029  0.008367       4        1   9.243258
15     0.007766  0.022229       5        1   8.008041
12    -0.008105  0.006125       6        1   6.218900
7      0.056067 -0.003451       7        1   4.544396
1     -0.009701 -0.010506       8        1   3.905261
0      0.005084  0.010557       9        1   3.448530
8     -0.003853  0.001846      10        1   2.954449
11     0.013286  0.024856      11        1   2.878418
17     0.005303 -0.005633      12        1   2.828351
9      0.011959 -0.044272      13        1   1.481292
16    -0.007221 -0.000812      14        1   1.445765
4      0.017178  0.005087      15        1   1.331487
10     0.028048  0.004434      16        1   1.045591
14    -0.001648 -0.016390      17        1   0.733675
2      0.011590 -0.016000      18        1   0.503891, topic_info=                Term          Freq         Total Category  logprob  loglift
231        werknemer  23685.000000  23685.000000  Default  30.0000  30.0000
166        werkgever  16421.000000  16421.000000  Default  29.0000  29.0000
120          sociaal  13389.000000  13389.000000  Default  28.0000  28.0000
38         betreffen  11675.000000  11675.000000  Default  27.0000  27.0000
246  bedrijfstoeslag   7317.000000   7317.000000  Default  26.0000  26.0000
..               ...           ...           ...      ...      ...      ...
29            bedrag     19.600581   5066.314954  Topic18  -5.7708  -0.2642
78              loon     18.788951   4451.607171  Topic18  -5.8131  -0.1772
256              dag     19.210955   4884.465233  Topic18  -5.7909  -0.2478
152           volgen     18.580243   5280.353533  Topic18  -5.8243  -0.3591
354       voorwaarde     18.549497   4397.207621  Topic18  -5.8260  -0.1777

[1573 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
11542     11  0.360688  Aanrekening
5071       1  0.088127     Aanvraag
5071       2  0.110159     Aanvraag
5071       3  0.066095     Aanvraag
5071       4  0.352508     Aanvraag
...      ...       ...          ...
174       15  0.002601        zwaar
174       16  0.002601        zwaar
174       17  0.016351        zwaar
174       18  0.007061        zwaar
18354      4  0.525312         امات

[7037 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 14, 6, 16, 13, 8, 2, 1, 9, 12, 18, 10, 17, 5, 11, 15, 3])

In [75]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)


C:\Users\nicyj\AppData\Local\Temp\ipykernel_18340\3297788535.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3,0.4197,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werklieden, betreffen, min..."
1,1,3,0.8597,"werknemer, toepassing, werkgever, betreffen, w...","[sociaal, direction_generaal, relations_collec..."
2,2,6,0.9971,"werknemer, werkgever, sociaal, bedrijfstoeslag...","[afsluiten, aanvullend, comité, werkman, inste..."
3,3,6,0.9968,"werknemer, werkgever, sociaal, bedrijfstoeslag...","[afsluiten, aanvullend, comité, werkman, inste..."
4,4,3,0.5036,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werklieid, wijziging, geld..."
5,5,8,0.6113,"werknemer, werkgever, betreffen, toepassing, c...","[aanvullen, comité, werklieden, betreffen, fie..."
6,6,3,0.9895,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werkman, inning, genaamd, ..."
7,7,3,0.9904,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werklieden, betreffen, opl..."
8,8,3,0.9880,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werklieden, betreffen, gev..."
9,9,3,0.9909,"werknemer, toepassing, werkgever, betreffen, w...","[aanvullen, comité, werklieden, betreffen, tij..."


In [76]:
df_dominant_topic.shape

(3813, 5)

In [78]:
df_dominant_topic.to_csv("C:/Users/nicyj/becode/KPMG/NEW/topicmodelling.csv", index = True)